# Merge and clean databases
## Libraries

In [1]:
# General libraries
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize

## Spain data
### Most frequent names by year
I download the data of the [100 most frequent names from 2002 to 2019 in Spain](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177009&menu=resultados&idp=1254734710990#!tabs-1254736195454), shared by the Instituto Nacional de Estadistica.

In [2]:
# Parameters
my_folder = "../data/raw/Spain/"
years = np.arange(2002, 2020)

# Open the files
df_sp = pd.DataFrame()
aux = pd.DataFrame()
for y in years:
    for i in ['nacional', 'Nacional', 'NACIONAL', 'TOTAL']:
        for j in ['.xls', '.xlsx']:
            try:
                aux = pd.read_excel(my_folder+'nomnac'+str(y)[-2:]+j, 
                                    sheet_name=i,
                                    skiprows=4, 
                                    header=None
                                   )
                try:
                    aux.columns = ['name_m', 'number_m', 'erase', 'name_f', 'number_f']
                except:
                    aux.columns = ['name_m', 'number_m', 'erase_1', 'name_f', 'number_f', 'erase_2']
            except:
                next
    aux['year'] = y
    aux = aux[['name_m', 'number_m', 'name_f', 'number_f', 'year']]
    df_sp = pd.concat([df_sp, aux])

In [3]:
# Wrangling data
# df_sp.drop(columns=["erase"])
males = df_sp[['name_m', 'number_m', 'year']]\
    .rename(columns={'name_m':'name', 'number_m': 'number'})
males['sex'] = "M"
females = df_sp[['name_f', 'number_f', 'year']]\
    .rename(columns={'name_f':'name', 'number_f': 'number'})
females['sex'] = "F"
df = pd.concat([females, males]).reset_index(drop=True)

# Names in lowercase
df.name = [str(n).lower() for n in df.name]

# Clean names
df.name = [re.sub(r'^\s+([A-Za-z])', r'\1', name) for name in df.name]
df.name = [re.sub(r'([A-Za-z0-9]+\s?[A-Za-z0-9]+)\s*$', r'\1', name) for name in df.name]
df = df[~((df.name=='total') | (df.name=='nan') | (df.name==' '))]
df['number'] = df['number'].astype(int)

# # Names in tokens
# df['tokens'] = [word_tokenize(n) for n in df.name]

In [4]:
# Save the data to CSV file
df.to_csv('../data/names/names_spain.csv')

### Average age by name
I download the data with [all names with frequency equal or higher to 20 people](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177009&menu=resultados&idp=1254734710990#!tabs-1254736195454), shared by the Instituto Nacional de Estadistica.

In [4]:
# Open the file
my_dic = {'Hombres':"M",
          "Mujeres":"F"}

aux = dict()
for i, j in my_dic.items():
    aux[i] = pd.read_excel(my_folder+'nombres_por_edad_media.xls', 
                  sheet_name=i,
                  skiprows=6 ,
                  names=['orden', 'name', 'number', 'average_age']
                 ).drop(columns=['orden'])
    aux[i]['sex'] = j

In [5]:
# Concatenate male and female databases
df = pd.concat([aux['Hombres'], aux['Mujeres']]).reset_index(drop=True)

# Names in lowercase
df.name = [str(n).lower() for n in df.name]

# Wrangling data
df.loc[30398 , 'name'] = 'na'

In [7]:
# Save the data to CSV file
df.to_csv('../data/names/names_spain_average_age.csv')

In [8]:
df.head()

,name,number,average_age,sex
0,antonio,666584,56.3,M
1,manuel,581915,55.2,M
2,jose,578937,61.5,M
3,francisco,488901,57.8,M
4,david,366782,31.2,M


## USA data
### Most frequent names by year
Names in [the US with frequency equal or higher to 5 observations per year](https://www.ssa.gov/oact/babynames/limits.html), by Social Security Administration.

In [11]:
# Parameters
my_folder = "../data/raw/USA/"
years = np.arange(1880, 2020)

# Open the file
df = pd.DataFrame()
for y in years:
    aux = pd.read_csv(my_folder + 'yob' + str(y) + '.txt', 
                     header=None).rename(columns={0:'name', 1:'sex', 2:"number"})
    aux['year'] = y
    df = pd.concat([df, aux]).reset_index(drop=True)

# Names in lowercase
df.name = [i.lower() for i in df.name]

In [12]:
# Save the data to CSV file
df.to_csv('../data/names/names_usa.csv')

## Canada data
### Most frequent names by year
In the case of Canada, I only found information from [British Columbia's Most Popular Baby Names](https://www2.gov.bc.ca/gov/content/life-events/statistics-reports/bc-s-most-popular-baby-names), shared by the 
Government of British Columbia.

In [14]:
# Parameters
my_folder = "../data/raw/Canada/"
my_dic = {'boys':"M",
          "girls":"F"}

aux = dict()
for i, j in my_dic.items():
    aux[i] = pd.read_csv(my_folder + 'bc-popular-' + i + '-names.csv', encoding ='latin1')
#     print(my_folder + 'bc-popular-' + i + '-names.csv')
    aux[i] = pd.DataFrame(aux[i].set_index('Name').stack()).reset_index()\
                    .rename(columns={'Name':'name', 'level_1':'year', 0:'number'})
    aux[i]['sex'] = j
    
# Concatenate male and female databases
df = pd.concat([aux['boys'], aux['girls']]).reset_index(drop=True)

# Names in lowercase
df.name = [str(n).lower() for n in df.name]

# Clean names
df = df[~(df.year=='Total')]
df.year = df.year.astype(int)

In [15]:
# Save the data to CSV file
df.to_csv('../data/names/names_canada.csv')